In [ ]:
import os, glob
from os import path

content_names = list(os.path.splitext(
        each)[0] for each in os.listdir("./input/contents"))
content_paths = glob.glob('./input/contents/*')

test = [path.splitext(path.basename(x))[0] for x in glob.glob('./input/contents/*')]
print(test)

# print(content_names, content_paths)

In [ ]:
import numpy as np
from PIL import Image
from os import listdir, mkdir, getcwd
import glob
import os

def import_image(file_name, height, width):
    # import image, cast as np array

    image = Image.open(file_name)  # 1.0 retrieve all input image references
    image = image.resize((height, width), Image.ANTIALIAS)  # downsample
    image = np.asarray(image, dtype='float32')  # cast to np array
    image = np.expand_dims(image, axis=0)  # add placeholder dimension

    image[:, :, :, 0] -= 103.939  # subtract mean RGB values from ImageNet
    image[:, :, :, 1] -= 116.779
    image[:, :, :, 2] -= 123.68
    image = image[:, :, :, ::-1]  # flip RGB pixel ordering

    return image


def export_image(img_array, save_dir, height, width):
    # export np array as jpeg image

    img_array = img_array.reshape((height, width, 3))  # reshape
    img_array = img_array[:, :, ::-1]  # flip RGB pixel ordering
    img_array[:, :, 0] += 103.939  # add mean RGB values from ImageNet
    img_array[:, :, 1] += 116.779
    img_array[:, :, 2] += 123.68
    img_array = np.clip(img_array, 0, 255).astype('uint8')  # trim pixels

    Image.fromarray(img_array).save(save_dir + '.jpeg', "jpeg")  # save


def retrieve_inputs():
    # retrieve all style and content image file names/dir paths

    content_names = [os.path.splitext(os.path.basename(x))[0]
                     for x in glob.glob('./input/contents/*')]
    content_paths = glob.glob('./input/contents/*')
    style_names = [os.path.splitext(os.path.basename(x))[0]
                   for x in glob.glob('./input/styles/*')]
    style_paths = glob.glob('./input/styles/*')

    return content_names, content_paths, style_names, style_paths


In [ ]:
import numpy as np
from keras import backend as K

def eval_loss_and_grads(x, loss, grads, composition_np, height, width):
    '''
    Define gradients of the total loss relative to the composition image, in
    order to minimize the loss.
    '''

    x = x.reshape((1, height, width, 3))
    outputs = [loss]
    outputs += grads
    f_outputs = K.function([composition_np], outputs)

    outs = f_outputs([x])
    prog_loss = outs[0]
    grad_values = outs[1].flatten().astype('float64')

    return prog_loss, grad_values


class Evaluator(object):
    '''
    Computes the loss and gradient present within the composition image. Phrased
    as a class to conveniently package retrieval methods for interfacing with 
    scypy.optimize library.
    '''

    def __init__(self, il, ig, comb_np, height, width):
        self.prog_loss = None  # bundle variables as object fields.. eek
        self.prog_grads = None
        self.initial_loss = il
        self.initial_grad = ig
        self.comb_np = comb_np
        self.height = height
        self.width = width

    def loss(self, x):
        '''
        Retrieve the loss value for the composition image.
        '''

        assert self.prog_loss is None
        self.prog_loss, self.grad_values = eval_loss_and_grads(x, self.initial_loss,
                                                               self.initial_grad,
                                                               self.comb_np,
                                                               self.height,
                                                               self.width)

        return self.prog_loss

    def grads(self, x):
        '''
        Retrieve the gradients associated with a particular composition
        image.
        '''

        assert self.prog_loss is not None
        grad_values = np.copy(self.grad_values)
        self.prog_loss = None
        self.grad_values = None

        return grad_values

In [ ]:
import numpy as np
import time
import scipy

from keras.applications.vgg16 import VGG16  # "OxfordNet", pretrained network
from keras import backend as K

from evaluator import Evaluator
from util import export_image


def content_loss(content_np, composition_np):
    '''
    Calculate and return the content loss between the content image and the
    combination image (Scaled Euclidean distance).
    '''

    return K.sum(K.square(composition_np - content_np))


def gram_matrix(image_np):
    '''
    Captures aggregate information about a particular image whilst ignoring 
    internal, structural detail.
    '''

    features = K.batch_flatten(K.permute_dimensions(image_np, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))

    return gram


def style_loss(style_np, composition_np, height, width):
    '''
    Calculate style loss between style reference image and combination image.
    Calculated as the scaled Frobenius norm of the difference between the Gram
    matrices of the style/combination images.
    '''

    S = gram_matrix(style_np)
    C = gram_matrix(composition_np)
    channels = 3
    size = height * width

    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def total_variation_loss(image_np, height, width):
    '''
    Reduce the noise present within the combined image by encnouraging spatial
    smoothness, achieved via regularization.
    '''

    a = K.square(image_np[:, :height - 1, :width - 1,
                          :] - image_np[:, 1:, :width - 1, :])
    b = K.square(image_np[:, :height - 1, :width - 1, :] -
                 image_np[:, :height - 1, 1:, :])

    return K.sum(K.pow(a + b, 1.25))


def total_loss(model, composition_np, content_weight, style_weight,
               height, width, total_variation_weight):
    '''
    Return the total loss present within the combination image.
    Total loss is calculated by considering the content, style and variation
    loss.
    '''

    loss = K.variable(0.)

    layers = dict([(layer.name, layer.output) for layer in model.layers])
    layer_features = layers['block2_conv2']
    content_features = layer_features[0, :, :, :]
    composition_features = layer_features[2, :, :, :]

    # content loss
    loss += content_weight * \
        content_loss(content_features, composition_features)

    # style loss calculation, retrieve all layers available for manipulation
    feature_layers = ['block1_conv2', 'block2_conv2',
                      'block3_conv3', 'block4_conv3',
                      'block5_conv3']

    for layer_name in feature_layers:
        layer_features = layers[layer_name]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_features, combination_features, height, width)
        loss += (style_weight / len(feature_layers)) * sl

    # total variation loss
    loss += total_variation_weight * \
        total_variation_loss(composition_np, height, width)
    return loss  # return total loss


def minimize_loss(save_name, loss, grads, composition_np, iterations, height, width):
    '''
    Using stochastic gradient descent via fmin_l_bfgs_b algorithm. Minimize
    and balance the loss experienced over the course of 10 iterations.
    Save the intermediate image combinations.
    '''

    x = np.random.uniform(0, 255, (1, height, width, 3)) - 128
    evaluator = Evaluator(loss, grads, composition_np, height, width)

    print("\n\nProcessing: " + save_name)
    for i in range(iterations):

        # print diagnostic information
        print('Start of iteration', i)
        start_time = time.time()
        x, min_val, info = scipy.optimize.fmin_l_bfgs_b(evaluator.loss,
                                                        x.flatten(),
                                                        fprime=evaluator.grads,
                                                        maxfun=20)
        print('Current loss value:', min_val)
        end_time = time.time()
        print('Iteration %d completed in %ds' % (i, end_time - start_time))
    return x


def create_composition_single(style_np, content_np, save_name, save_dir,
                              height, width, content_weight, style_weight,
                              total_variation_weight, iterations):
    '''
    Combine all components, create composite image with supplied
    content and style image arrays.
    '''

    # placeholder composition image
    composition_np = K.placeholder((1, height, width, 3))

    # concat image arrays as single tensor
    input_tensor = K.concatenate([content_np,
                                  style_np,
                                  composition_np], axis=0)

    # load model
    model = VGG16(input_tensor=input_tensor,
                  weights='imagenet', include_top=False)

    # define initial loss
    loss = total_loss(model, composition_np, content_weight,
                      style_weight, height, width,
                      total_variation_weight)

    # define initial gradients
    grads = K.gradients(loss, composition_np)

    # run optimization using previously calculated loss values
    composition = minimize_loss(save_name, loss, grads,
                                composition_np, iterations,
                                height, width)

    # 5.0 convert and finalize np array
    export_image(composition, save_dir, height, width)

In [ ]:
from os import mkdir
from util import retrieve_inputs, import_image
from style_transfer import create_composition_single


def main():

    # parameterize composition model
    content_weight = 0.025
    style_weight = 5.0
    variation_weight = 1.0
    height = 50  # image downsample dimensions, min of 48x48 required
    width = 50
    iterations = 1

    # retrieve all inputs
    content_names, content_paths, style_names, style_paths = retrieve_inputs()

    # for every content image
    for cn, cp in zip(content_names, content_paths):
        mkdir('./output/' + cn)
        content_np = import_image(cp, height, width)

        # create all possible content/style composition images
        for sn, sp in zip(style_names, style_paths):
            style_np = import_image(sp, height, width)
            save_name = cn + '_' + sn
            save_dir = './output/' + cn + '/' + save_name

            create_composition_single(style_np, content_np,
                                      save_name, save_dir,
                                      height, width, content_weight,
                                      style_weight, variation_weight,
                                      iterations)

main()